In [1]:
import cv2
import dlib
import numpy as np

In [2]:
#For detecting faces
detector = dlib.get_frontal_face_detector() 
landmark_path="shape_predictor_68_face_landmarks.dat" 
#For identifying landmarks
predictor = dlib.shape_predictor(landmark_path) 

In [3]:
#Obtaining Facial Landmark coordinates
def get_facial_landmarks(image):
    face = detector(image, 1)
    #Detecting faces in image
    if len(face) > 1:
        return "Multiple faces detected in the frame!!"
    if len(face) == 0:
        return "No face detected in the frame!!"
    #Return the coordinates
    return np.matrix([[pred.x, pred.y] for pred in predictor(image, face[0]).parts()])

In [4]:
#Drawing the landmarks : yellow in color 
def landmarks_annotation(image, facial_landmarks):
    
    image = image.copy()
    for coord, p in enumerate(facial_landmarks):
        position = (p[0, 0], p[0, 1])
        cv2.putText(image, str(coord), position, cv2.FONT_HERSHEY_COMPLEX, 0.3, (0, 255, 255))
    return image

In [5]:
#Landmark coordinates for upper lip identified in the face 
def upperlip(facial_landmarks):
    ulip = []
    #create an array to store the landmark coordinates of the upper lip
    for i in range(50,53):
        ulip.append(facial_landmarks[i])
    for i in range(61,64):
        ulip.append(facial_landmarks[i])
    
    ulip_mean = np.mean(ulip, axis=0)
    return int(ulip_mean[:,1])

In [6]:
#Landmark coordinates for lower lip identified in the face 
def lowerlip(facial_landmarks):
    llip = []
    #create an array to store the landmark coordinates of the lower lip
    for i in range(65,68):
        llip.append(facial_landmarks[i])
    for i in range(56,59):
        llip.append(facial_landmarks[i])
        
    llip_mean = np.mean(llip, axis=0)
    return int(llip_mean[:,1])#centroid value

In [7]:
#Detect the yawning activity
def yawning(image):
    #Obtain the facial Landmark coordinates
    facial_landmarks = get_facial_landmarks(image)
    if type(facial_landmarks) == str:
        return image, 0
    
    #Obtain the frame / image with annotated facial landmarks
    landmarks_image = landmarks_annotation(image, facial_landmarks)
    
    #Obtain Lip centroids
    upperlip_centroid = upperlip(facial_landmarks)
    lower_lip_centroid = lowerlip(facial_landmarks)
    
    #Calculate the distance between the centroids
    lips_dist = abs(upperlip_centroid - lower_lip_centroid)
    return landmarks_image, lips_dist

In [8]:
yawn_status = False 
yawn_count = 0

In [14]:
img = cv2.imread('dumb.jpg')
landmarks_image, lips_dist = yawning(img)

#Update the yawn status
previous_status = yawn_status  

#lips distance is subjective and changes from subject to subject based on their facial structures
if lips_dist > 47:
    
    yawn_status = True
    cv2.putText(img, "You are yawning", (50,450), cv2.FONT_HERSHEY_COMPLEX, 1,(0,255,255))
        
else:
    yawn_status = False 
    cv2.putText(img, "You are not yawning", (50,450), cv2.FONT_HERSHEY_COMPLEX, 1,(255,0,0))
        

cv2.imshow('Yawning Activity Detection', img )
cv2.waitKey(0)
cv2.destroyAllWindows()